# PARTE 1 - Processamento dos Dados - Kaggle - TalkingData

## Alunos: Ana Carolina Wagner e José Valentim
## Mestrado FGV-EMAp - 2018.1
## MMD - Prof. Renato

## Fontes:

* [Kaggle Notebook - How to Work with BIG Datasets on Kaggle Kernels](https://www.kaggle.com/yuliagm/how-to-work-with-big-datasets-on-16g-ram-dask)
* [Kaggle Discussion - How to manage the amount of data](https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/51411)
* [O`Reilly - Jeroen Janssens - Data Science at the Command Line](https://www.datascienceatthecommandline.com/chapter-5-scrubbing-data.html) 
* [Working with CSVs on the Command Line](http://bconnelly.net/working-with-csvs-on-the-command-line/)
* [Pandas on Dask](http://dask.pydata.org/en/latest/dataframe.html) (`import pandas as pd < import ray.dataframe as pd < import dask.dataframe as dd`)

## 1. Carregando bibliotecas 

In [1]:
import pandas as pd
import dask.dataframe as dd
import numpy as np

import os
import csv
import time
import gc # garbage collection 

path = '../../../../../dados/Dados/Kaggle/' # Servidor - Renato 
path_ana = os.getcwd()

## 2. Importando os dados

In [2]:
print('# Tamanho dos dados:')
for f in os.listdir(path)[1:8]:
    print(f.ljust(30) + str(round(os.path.getsize(path  + f) / 1000000, 2)) + ' MB')

# Tamanho dos dados:
sample_submission.csv         195.58 MB
test.csv.zip                  169.8 MB
train.csv.zip                 1298.59 MB
train_sample.csv.zip          1.13 MB
sample_submission.csv.zip     42.06 MB
train.csv                     7537.65 MB
train_sample.csv              4.08 MB


Analisando os dados da competição, podemos perceber que se trata de um grande volume de dados. Portanto, torna-se necessário algumas manipulações para que possamos trabalhar com os dados de treinamento (o arquivo `train.csv` possui mais de 180 milhões de linhas!!!).

In [3]:
df_train_sample = pd.read_csv(path + 'train_sample.csv')
df_test = pd.read_csv(path + 'test.csv')

In [4]:
df_train_sample.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0


In [5]:
df_train_sample[df_train_sample['attributed_time'].notnull()].head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
284,224120,19,0,29,213,2017-11-08 02:22:13,2017-11-08 02:22:38,1
481,272894,10,1,7,113,2017-11-08 06:10:05,2017-11-08 06:10:37,1
1208,79001,19,0,0,213,2017-11-07 09:54:22,2017-11-07 11:59:05,1
1341,131029,19,0,0,343,2017-11-09 10:58:46,2017-11-09 11:52:01,1
1412,40352,19,0,0,213,2017-11-07 22:19:03,2017-11-08 01:55:02,1


In [6]:
df_train_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
ip                 100000 non-null int64
app                100000 non-null int64
device             100000 non-null int64
os                 100000 non-null int64
channel            100000 non-null int64
click_time         100000 non-null object
attributed_time    227 non-null object
is_attributed      100000 non-null int64
dtypes: int64(6), object(2)
memory usage: 6.1+ MB


In [7]:
df_test.head()

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00


In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18790469 entries, 0 to 18790468
Data columns (total 7 columns):
click_id      int64
ip            int64
app           int64
device        int64
os            int64
channel       int64
click_time    object
dtypes: int64(6), object(1)
memory usage: 1003.5+ MB


A princípio, podemos observar que a coluna `attributed_time` agrega pouca informação adicional ao restante dos dados. Trata-se de um timestamp do horário em que o download foi realizado, dessa forma ele é gerado após o clique. Como estamos querendo classificar um determinado clique, essa informação foi julgada ser redundante para prever se o download de fato ocorrerá (ela também não está presente no conjunto teste, pois está diretamente relacionada com a coluna que queremos fazer a predição). Dessa forma iremos descartar essa coluna na análise exploratória dos dados. 

In [9]:
print('Dados de treinamento:')
print('     Mínimo click_time: ' + str(min(df_train_sample['click_time'])))
print('     Máximo click_time: ' + str(max(df_train_sample['click_time'])))
print('Dados de teste:')
print('     Mínimo click_time: ' + str(min(df_test['click_time'])))
print('     Máximo click_time: ' + str(max(df_test['click_time'])))

Dados de treinamento:
     Mínimo click_time: 2017-11-06 16:00:00
     Máximo click_time: 2017-11-09 15:59:51
Dados de teste:
     Mínimo click_time: 2017-11-10 04:00:00
     Máximo click_time: 2017-11-10 15:00:00


Como podemos observar, a coluna `click_time` é um [timestamp](https://en.wikipedia.org/wiki/Timestamp) no formato **"yyyy-mm-dd hh:mm:s"**. Em ambos datasets, os dados dessa coluna oscilam em um intervalo de `2017-11-06` a `2017-11-10` (4 dias). Ou seja, as informações sobre mês e ano são redundantes (todos registros são de **novembro de 2017**. 

A ideia é carregar os datasets descartando as informações sobre mês e ano. Para isso, vamos separar esse timestamp em colunas e considerar apenas: `day`, `hour`, `minute` e `second`.

In [10]:
# Importando Apenas As Colunas Selecionadas
train_cols = ['ip', 'app', 'device', 'os', 'channel', 'day', 'hour','is_attributed']
test_cols  = ['ip', 'app', 'device', 'os', 'channel', 'day', 'hour']

In [11]:
# Data Types - reduzir o espaço de armazenamento usando inteiros onde possível
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'day'           : 'uint8',
        'hour'          : 'uint8',
        'minute'        : 'uint8',
        'second'        : 'uint8'
        }

Em vez de deixar que o Python adivinhe os tipos de dados, em que acabará colocando mais espaço do que necessário, sabendo que os números são inteiros e que não ultrapassam determinados valores, vamos definir os tipos de dados com os requisitos mínimos antes de importá-los. 

A partir das considerações feitas e pesqusiando no campo de [Discussion](https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion) do Kaggle, resolvemos tratar algumas dessas modificações propostas na linha de comando e salvar os resultados em um novo `csv`.

[**TIP1: Random selection of all the train set**](https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/51809#295230) Optamos por trabalhar com uma amostra de 10% dos dados do conjunto de treinamento. Para isso, utilizamos o `Pandas On Dask`.

In [12]:
# Inicia o processamento do conjunto de treinamento usando o Pandas On Dask - mais rápido que o Pandas tradicional
# start_time = time.time()
# print('[{}] Start to load data'.format(time.time() - start_time))
# freq = 0.1 # Porcentagem da amostra do conjunto de treinamento
# df_train = df_train.random_split([freq , 1-freq])[0]
# df_train = df_train.compute()
# print('[{}] Finished to load data'.format(time.time() - start_time))

In [13]:
# Salva a amostra gerada do conjunto de treinamento 
#df_train.to_csv(path_ana + '/datasets/train_sample_10.csv')

[**TIP2: Shrink the data from command line**](https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/51347) Com a amostra gerada, vamos remover as informações de ano e mês da coluna `click_time` na linha de comando. Fazemos o mesmo para o conjunto teste original.

* Para o conjunto de treinamento: `sed s/2017-11-//g < train_sample_10.csv > train_sample_10_reduced.csv`
* Para o conjunto teste: `sed s/2017-11-//g < test.csv > test_reduced.csv`

Em seguida,

* Para o conjunto de treinamento: sed `-E -e 's/[[:space:]]/,/g' -e 's/:/,/g' -e 's/click_time/day,hour,minute,second/g' < train_sample_10_reduced.csv > train_sample_10_reduced.csv`
* Para o conjunto teste: `sed -E -e 's/[[:space:]]/,/g' -e 's/:/,/g' -e 's/click_time/day,hour,minute,second/g' < test_reduced.csv > test_reduced.csv`

In [16]:
df_train_reduced = pd.read_csv(path_ana + "/datasets/train_sample_10_reduced.csv", dtype=dtypes, usecols = train_cols)
df_test_reduced = pd.read_csv(path_ana + "/datasets/test_reduced.csv", dtype=dtypes, usecols = test_cols)

In [17]:
df_train_reduced.head()

,ip,app,device,os,channel,day,hour,is_attributed
0,35810,3,1,13,379,6,14,0
1,165970,3,1,13,379,6,14,0
2,111385,3,1,18,379,6,14,0
3,74715,3,1,19,379,6,14,0
4,148454,3,1,19,379,6,14,0


In [18]:
df_test_reduced.head()

,ip,app,device,os,channel,day,hour
0,5744,9,1,3,107,10,4
1,119901,9,1,3,466,10,4
2,72287,21,1,19,128,10,4
3,78477,15,1,13,111,10,4
4,123080,12,1,13,328,10,4
